In [0]:
import numpy as np

from sympy.integrals.quadrature import gauss_lobatto
from scipy.optimize import fsolve

In [0]:
from IPython.display import clear_output
import timeit

# Nodi di interpolazione

La funzione **generate_interpolation_nodes** restituisce i nodi di interpolazione dull'elemento di riferimento $\hat{K}=[-1,1]^d$.

Gli argomenti da fornirle sono due:
- $n$: un intero maggiore di 1 che indica quanti sono i punti di Chebyshev-Gauss-Lobatto che vogliamo considerare per generare i nodi;
- $d$: un intero a scelta tra 2 e 3 che indica la dimensione dello spazio $\mathbb{R}^d$ all'interno del quale si trova $\hat{K}$.

La funzione restituisce in output un array di dimensione $(n^d, d)$.

In [0]:
def generate_interpolation_nodes(n, d):
    
    assert d==2 or d==3
    assert n>1

    def generate_CGL_nodes(n):
        q, w = gauss_lobatto(n, 10)
        q = np.array(q, dtype=float)
        return q

    def generate_interpolation_nodes_2d(n):
        q = generate_CGL_nodes(n)
        Q1, Q2 = np.meshgrid(q,q)
        grid = np.stack([Q1,Q2], axis=-1).reshape((-1,2))
        return grid
        
    def generate_interpolation_nodes_3d(n):
        q = generate_CGL_nodes(n)
        Q1, Q2, Q3 = np.meshgrid(q,q,q)
        grid = np.stack([Q1,Q2,Q3], axis=-1).reshape((-1,3))
        return grid

    if(d==2):
      return generate_interpolation_nodes_2d(n)
    elif(d==3):
      return generate_interpolation_nodes_3d(n)
    else:
      print("Dimension d must be 2 or 3")
      return

# Celle random

La funzione **generate_standard_cell** costruisce i punti di supporto di una cella standard $K$ distorcendo i punti di interpolazione di $\hat{K}$.

Gli argomenti da fornirle sono quattro:
- $n$: un intero maggiore di 1 che indica quanti sono i punti di Chebyshev-Gauss-Lobatto che vogliamo considerare per generare i nodi di interpolazione di $\hat{K}$;
- $d$: un intero a scelta tra 2 e 3 che indica la dimensione dello spazio $\mathbb{R}^d$ all'interno del quale si trova $K$;
- $\textit{eps}$: un numero che indica il grado di distorsione da applicare ai punti di interpolazione;
- $\textit{theta}$: rotazione a cui sarà sottoposta la cella distorta (se $d=2$ deve essere un numero, se $d=3$ allora dovrà essere una lista di tre numeri.

La funzione restituisce in output un array di dimensione $(n^d, d)$.

In [0]:
def generate_standard_cell(n, d, eps=0, theta=0):
    
    assert n>1
    assert d==2 or d==3

    #Distortion
    K = generate_interpolation_nodes(n,d)
    distortion = (np.random.random(K.shape)-.5)*eps
    K += distortion
    
    theta = np.array(theta)
    #2D rotation
    if(d==2):
      assert len(theta.shape)==0
      R = np.array([[np.cos(theta), np.sin(theta)],[-np.sin(theta), np.cos(theta)]])
      K = np.tensordot(K, R, axes=[[-1],[-1]])
    #3D rotation
    elif (d==3):
      assert theta.shape[0]==3
      a, b, c = theta[0], theta[1], theta[2]
      yaw = np.array([[np.cos(a),-np.sin(a),0.],[np.sin(a),np.cos(a),0.],[0.,0.,1.]])
      pitch = np.array([[np.cos(b),0.,np.sin(b)],[0.,1.,0.],[-np.sin(b),0.,np.cos(b)]])
      roll = np.array([[1.,0.,0.],[0.,np.cos(c),-np.sin(c)],[0.,np.sin(c),np.cos(c)]])
      R = np.matmul(np.matmul(yaw,pitch),roll)
      K = np.tensordot(K, R, axes=[[-1],[-1]])
      
    #Standardization
    m, M = np.min(K, axis=0), np.max(K, axis=0)
    h = M-m
    K -= m
    K /= h/2
    K -= 1
    return K

# Polinomi di Lagrange

La funzione **generate_L** costruisce la funzione $\mathcal{L}_K: \mathbb{R}^d \longrightarrow \mathbb{R}^d$ che mappa l'elemento di supporto $\hat{K}$ nella cella $K$.
L'unico argomento da fornirle sono i punti di supporto di $K$, ottenuti dalla funzione $\textit{generate_standard_cell}$, e in automatico viene costruita la funzione e restituita come output.

La funzione restituita è vettorizzata:


```
K = generate_standard_cell(3,2,.5,1)
L = generate_L(K)
L([[0,0]]) # restituisce l'immagine del punto [0,0]
L([[0,0],[5,2],[3,4]]) # restituisce un array contenente le immagini dei tre punti
```




In [0]:
def generate_L(K):
   
    d = K.shape[1]
    n = int(pow(K.shape[0],1/d)+.5)
    Khat = generate_interpolation_nodes(n,d)
    q, w = gauss_lobatto(n, 10)
    nodes = np.array(q, dtype=float)

    def Lagrange(i,x):
        x = x.repeat(n-1).reshape(-1,d*(n-1))
        v = []
        for k in range(d):
            v.append(np.delete(nodes,np.where(nodes==Khat[i][k])))
        v = np.array(v).flatten()
        v = v.repeat(x.shape[0]).reshape((-1,len(v)), order='F')
        avoid = Khat[i].repeat(n-1).repeat(x.shape[0]).reshape(-1,d*(n-1), order='F')
        return np.prod((x-v)/(avoid-v), axis=1)
    
    def L(x):
        x = np.array(x)
        lgr = np.zeros((0,x.shape[0]))
        for i in range(len(Khat)):
            lgr = np.append(lgr, Lagrange(i,x))
        lgr = np.stack(lgr.reshape((-1,x.shape[0])),axis=-1)
        return np.matmul(lgr,K)

    return L

# Full inversion dataset

La funzione **dataset_full** genera il dataset per l'inversione completa. Gli argomenti da fornirle sono sei:
- $n$: un intero maggiore di 1 che indica quanti sono i punti di Chebyshev-Gauss-Lobatto che vogliamo considerare per generare i nodi di interpolazione di $\hat{K}$;
- $d$: un intero a scelta tra 2 e 3 che indica la dimensione dello spazio $\mathbb{R}^d$ all'interno del quale si trova $K$;
- $\textit{distortions}$: una lista di numeri utilizzati per la distorsione delle celle random;
- $\textit{angles}$: una lista di numeri per $d=2$ o una lista di triplette per $d=3$, utilizzati per la rotazione delle celle random;
- $\textit{delta}$: ampiezza dell'intorno che scegliamo per $\hat{K}$ (default $0.5$);
- $\textit{n_sampling_points}$: numero con cui costruiamo la griglia di punti di cui vogliamo calcolare le immagini attraverso le funzioni $\mathcal{L}_K$.

La funzione restituisce in output due array: $\textit{xtrain}$ e $\textit{ytrain}$.


In [0]:
def dataset_full(n, d, distortions, angles, delta=.5, n_sampling_points=100):
    assert n==2 or n==3 or n==4
    assert d==2 or d==3
    distortions = np.array(distortions)
    angles = np.array(angles)
    if (d==3):
       assert angles.shape[1]==3

    xhat_1d = np.linspace(-delta-1,1+delta,n_sampling_points)
    if(d==2):
      xhat = np.stack(np.meshgrid(xhat_1d,xhat_1d), axis=-1).reshape((-1,d))
    else:
      xhat = np.stack(np.meshgrid(xhat_1d,xhat_1d,xhat_1d), axis=-1).reshape((-1,d))

    xtrain = np.zeros((0,(pow(n,d)+1)*d))
    ytrain = np.zeros((0,d))

    i=0
    print("FULL INVERSE DATASET")
    print("Percentuale di completamento: 0%")
    start = timeit.default_timer()

    for A in distortions:
        i+=1
        clear_output(wait=True)
        for B in angles:
            

            K = generate_standard_cell(n,d,A,B)
            L = generate_L(K)
            x = L(xhat)
            mask = (x >= -1).all(axis=-1) * (x <= 1).all(axis=-1)
            x = x[mask,:]
            xhat = xhat[mask,:]
            
            xdata = np.concatenate((x, np.reshape(np.repeat(K.flatten(),x.shape[0]), (x.shape[0],-1) ,order='F')), axis=-1)

            xtrain = np.append(xtrain, xdata, axis=0)
            ytrain = np.append(ytrain, xhat[...,:], axis=0)

        stop = timeit.default_timer()
        time = np.round((stop-start)/60 , 2)
        wait = np.round((time/i)*(len(distortions)-i), 2)
        print("FULL INVERSE DATASET")
        print("Tempo trascorso: ", time, " minuti")
        print("Tempo rimanente stimato: ", wait, " minuti")
        print("Percentuale di completamento: ", np.round((i/len(distortions))*100,2), "%") 


    return xtrain, ytrain

# Polynomial inversion dataset

La funzione **find_polynomial_inverse** costruisce i punti di supporto per l'inversione polinomiale. Gli argomenti da fornirle sono due:
- $K$: l'insieme dei punti di supporto della cella $K$ costruiti attraverso $\textit{generate_standard_cell}$;
- $\textit{up}$: un intero che indica di quanto vogliamo aumentare il grado dell'inversa rispetto al grado di $\mathcal{L}_K$.

In [0]:
def find_polynomial_inverse(K,up):
    d = K.shape[1]
    n = int(pow(K.shape[0],1/d)+.5) + up
    Khat = generate_interpolation_nodes(n,d)

    def residual(V):
        V = V.reshape((-1,d))
        F = generate_L(K)
        res = F(V)- Khat
        return res.flatten()
    
    return fsolve(residual,Khat.flatten()).reshape(-1,d)

La funzione **dataset_polynomial** genera il dataset per l'inversione polinomiale. Gli argomenti da fornirle sono cinque:
- $n$: un intero maggiore di 1 che indica quanti sono i punti di Chebyshev-Gauss-Lobatto che vogliamo considerare per generare i nodi di interpolazione di $\hat{K}$;
- $\textit{up}$: un intero che indica di quanto vogliamo aumentare il grado delle inverse rispetto al grado delle $\mathcal{L}_K$;
- $d$: un intero a scelta tra 2 e 3 che indica la dimensione dello spazio $\mathbb{R}^d$ all'interno del quale si trovano le celle $K$;
- $\textit{distortions}$: una lista di numeri utilizzati per la distorsione delle celle random;
- $\textit{angles}$: una lista di numeri per $d=2$ o una lista di triplette per $d=3$, utilizzati per la rotazione delle celle random.

La funzione restituisce in output due array: $\textit{xtrain}$ e $\textit{ytrain}$.

In [0]:
def dataset_polynomial(n, up, d, distortions, angles):
    assert n==2 or n==3 or n==4
    assert d==2 or d==3
    
    
    distortions = np.array(distortions)
    angles = np.array(angles)
    if (d==3):
       assert angles.shape[1]==3

    xtrain = np.zeros((0,(pow(n,d)*d)))
    ytrain = np.zeros((0,(pow(n+up,d)*d)))

    i=0
    print("POLYNOMIAL DATASET")
    print("Percentuale di completamento: 0%")
    start = timeit.default_timer()


    for A in distortions:

        i+=1
        clear_output(wait=True)

        for B in angles:
            K = generate_standard_cell(n,d,A,B)
            xtrain = np.append(xtrain, K.flatten())
            ytrain = np.append(ytrain, find_polynomial_inverse(K,up))

        stop = timeit.default_timer()
        time = np.round((stop-start)/60 , 2)
        wait = np.round((time/i)*(len(distortions)-i), 2)
        print("POLYNOMIAL DATASET")
        print("Tempo trascorso: ", time, " minuti")
        print("Tempo rimanente stimato: ", wait, " minuti")
        print("Percentuale di completamento: ", np.round((i/len(distortions))*100,2), "%")    

    xtrain = xtrain.reshape((-1,(pow(n,d)*d)))
    ytrain = ytrain.reshape((-1,(pow(n+up,d)*d)))

    return xtrain, ytrain

# FULL DATASETS GENERATION

In [0]:
xtrain, ytrain = dataset_full(n = 2,
                              d = 2,
                              distortions = np.linspace(0,1,1000),
                              angles = np.linspace(0,2*np.pi,360),
                              delta = .5,
                              n_sampling_points = 250)
np.save("X_full_2_2", xtrain)
np.save("Y_full_2_2", ytrain)

In [0]:
xtrain, ytrain = dataset_full(n = 3,
                              d = 2,
                              distortions = np.linspace(0,.5,500),
                              angles = np.linspace(0,2*np.pi,360),
                              delta = .5,
                              n_sampling_points = 250)
np.save("X_full_3_2", xtrain)
np.save("Y_full_3_2", ytrain)

In [0]:
xtrain, ytrain = dataset_full(n = 2,
                              d = 3,
                              distortions = np.linspace(0,1,1000),
                              angles = np.linspace(0,2*np.pi,360),
                              delta = .5,
                              n_sampling_points = 100)
np.save("X_full_2_3", xtrain)
np.save("Y_full_2_3", ytrain)

In [0]:
xtrain, ytrain = dataset_full(n = 3,
                              d = 3,
                              distortions = np.linspace(0,.5,500),
                              angles = np.linspace(0,2*np.pi,360),
                              delta = .5,
                              n_sampling_points = 100)
np.save("X_full_3_3", xtrain)
np.save("Y_full_3_3", ytrain)

# POLYNOMIAL DATASETS GENERATION

In [0]:
xtrain, ytrain = dataset_polynomial(n = 2,
                                    up = 1,
                                    d = 2,
                                    distortions = np.linspace(0,1,1000),
                                    angles = np.linspace(0,2*np.pi,360))
np.save("X_polynomial_2_1_2", xtrain)
np.save("Y_polynomial_2_1_2", ytrain)

In [0]:
xtrain, ytrain = dataset_polynomial(n = 3,
                                    up = 1,
                                    d = 2,
                                    distortions = np.linspace(0,.5,500),
                                    angles = np.linsapce(0,2*np.pi,360))
np.save("X_polynomial_3_1_2", xtrain)
np.save("Y_polynomial_3_1_2", ytrain)